In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, HTML

## ETL

In [ ]:
mens_df = pd.read_csv("data\ebay_mens_perfume.csv")
womens_df = pd.read_csv("data\ebay_womens_perfume.csv")

In [ ]:
mens_df.info()

In [ ]:
womens_df.info()

In [ ]:
mens_df['gender'] = 'man'
womens_df['gender'] = 'woman'
perfume_df = pd.concat ([mens_df, womens_df], axis = 0)
perfume_df.info()

In [ ]:
perfume_df = perfume_df.fillna({
    'brand': 'Unknown',
    'type': 'Unknown',
    'available': 0,
    'availableText': 'Not available',
    'sold': 0
})

In [ ]:
float_columns = perfume_df.select_dtypes(include=['float64']).columns
perfume_df[float_columns] = perfume_df[float_columns].astype(int)

In [ ]:
perfume_df = perfume_df.drop(['priceWithCurrency','availableText', 'lastUpdated'], axis = 1)

In [ ]:
category_mapping = {
    'Eau de Parfum': ['Eau de Parfum', 'Eau De Parfum', 'Eau de Parfum Intense', 'Eau De Parfum Intense',
                      'Eau De Parfum 2 Pcs Set', "L'Eau de Parfum", 'Eau de Parfume', "L'eau de Parfum", 
                      'EDP and Parfum', 'Eau De Parfum Supreme', 'Eau de Parfum/Perfume', 'Eau De Parfum Spray', 
                      'Eau de Parfum, Eau De Parfume', 'Eau de Parfum, Spray', 'Eau De Parfum 3 Pcs Set', 
                      'LE PARFUM', 'EXTRAIT DE PARFUM', 'Extrait De Parfum', 'Extrait de Parfum', 'Elixir de Parfum',
                      'Elixir De Parfum', 'Extract Parfum', 'Parfum Intense', 'Eau de Perfume'],
    
    'Eau de Toilette': ['Eau de Toilette', 'Eau De Toilette', 'Eau de Toilette Intense', 'Eau de Toillette', 
                        'Eau De Toilette, Eau De Parfum, Eau De Parfum Intense', 'Eau De Toilette Spray 3.4 oz',
                        'Eau de Toilette, Spray'],
    
    'Cologne': ['Cologne', 'Eau de Cologne', 'Eau De Cologne', 'Fine Cologne', 'Cologne Spray', 
                'Cologne spray', 'DIOR HOMME COLOGNE', 'Jo Malone Cologne Intense Spray', 'EAU DE COLOGNE SPRAY',
                'Eau de Cologne Spray, Cologne Spray'],
    
    'Body Spray': ['Body Spray', 'Fragrance Body Spray', 'Deodorant Body Spray', 'Body Mist', 'Fragrance Mist', 
                   'Fine Fragrance Mist', 'Perfume Fragrance Mist', 'Fragrance Rolling Ball', 'Deodorant Stick', 
                   'Deodorant', 'Deodorant Body Spray', 'Roll On'],
    
    'Perfume Oil': ['Perfume Oil', 'Fragrance Oil', 'Body Oil', 'Oil Perfume', 'Scented Oils', 'Concentrated Uncut Pure Body Oil',
                    'Oil', 'Shimmering Body Oil', 'Bath Oil']
}

In [ ]:
perfume_df['grouped_category'] = perfume_df['type'].apply(lambda value: next((category for category, keywords in category_mapping.items() if value in keywords), 'Other_product_type'))

In [ ]:
perfume_df['available_value'] = perfume_df['price']*perfume_df['available']
perfume_df['total_revenue'] = perfume_df['price']*perfume_df['sold']

In [ ]:
brands = perfume_df['brand'].unique()

In [ ]:
luxury = ['Chanel', 'Dior', 'Yves Saint Laurent', 'Gucci', 'Giorgio Armani',
            'Hermès', 'Prada', 'Versace', 'Valentino', 'Louis Vuitton',
            'Givenchy', 'Tom Ford', 'Burberry', 'Viktor & Rolf', 'Creed',
            'Maison Francis Kurkdjian', 'Bvlgari', 'Acqua di Parma', 'Lalique',
            'Dolce & Gabbana', 'Montale', 'Azzaro', 'Chanel No 5', 'Oscar de la Renta',
            'Hermès Paris', 'La Prairie', 'Guerlain', 'By Kilian', 'Celine',
            'Atelier', 'Chanel Paris', 'Giorgio Armani', 'Tommy Bahama', 'Kenzo',
            'Jean Paul Gaultier', 'Bvlgari', 'Dolce & Gabbana', 'Christian Dior',
            'Serge Lutens', 'Frederic Malle', 'Amouage', 'Xerjoff', 'Roja Parfums',
            'Penhaligon\'s', 'Juliette Has a Gun', 'Parfums de Marly', 'Nasomatto',
            'Fragonard', 'Mancera', 'Al Haramain', 'Alexander McQueen', 'Rasasi',
            'Montblanc', 'Carolina Herrera']

massmarket = [ 'AXE', 'Old Spice', 'David Beckham', 'Adidas', 'Coty',
            'Nautica', 'Kenneth Cole', 'Bath & Body Works',
            'Paris Hilton', 'Abercrombie & Fitch', 'Calvin Klein', 'Paco Rabanne',
            'Elizabeth Taylor', 'Jovan', 'Lacoste', 'Curve', 'Iman', 'Cacharel',
            'Giorgio Beverly Hills', 'Benetton', 'Avon', 'Gwen Stefani',
            'Hollister', 'Sbyd', 'Marlene Dietrich', 'Hugo Boss']

niche = ['Amouage', 'Byredo', 'Diptyque', 'Maison Francis Kurkdjian',
            'Montale', 'Kilian', 'Xerjoff', 'Roja Parfums', 'Juliette has a gun',
            "Penhaligon\'s", 'Parfums de Marly', 'Nasomatto', 'Frederic Malle',
            'Tiziana Terenzi', 'Serge Lutens', 'Fragonard',
            'Creed', 'Acqua di Parma', 'Lalique', 'The Different Company',
            'Vilhelm Parfumerie', "Eau d'Italie", 'Vero Profumo', 'Aesop', 'Diptyque']

indie = ['Maison Alhambra', 'Al Haramain', 'Clive Christian', 'Alexandria Fragrances',
            'Roja', 'The Baron', 'Juliette Has a Gun', 'Huda Beauty Kayali',
            'Atelier Cologne', 'Byredo', 'Aesop', 'Le Labo', 'Miller Harris',
            'DS & Durga', 'Guerlain', 'Parfums de Marly', 'Memo Paris', 'Mizensir']

In [ ]:
def classify_brand(brand):
    if brand in luxury:
        return 'Luxury'
    elif brand in massmarket:
        return 'Massmarket'
    elif brand in niche:
        return 'Niche'
    elif brand in indie:
        return 'Indie'
    else:
        return 'Other_category'

In [ ]:
perfume_df['category'] = perfume_df['brand'].apply(classify_brand)

In [ ]:
perfume_df.info()

In [ ]:
perfume_df.describe()

In [ ]:
perfume_df

In [ ]:
perfume_df.info()

In [ ]:
sql_perfume = perfume_df

In [ ]:

sql_perfume = sql_perfume.applymap(lambda x: x.strip() if isinstance(x, str) else x)

sql_perfume.replace(r'[\n\r]+', ' ', regex=True, inplace=True)
sql_perfume.replace('"', '', regex=True, inplace=True)

sql_perfume.fillna(0, inplace=True)

sql_perfume.drop_duplicates(inplace=True)

sql_perfume['price'] = sql_perfume['price'].astype('int64')
sql_perfume['available'] = sql_perfume['available'].astype('int64')
sql_perfume['sold'] = sql_perfume['sold'].astype('int64')
sql_perfume['available_value'] = sql_perfume['available_value'].astype('int64')
sql_perfume['total_revenue'] = sql_perfume['total_revenue'].astype('int64')

In [ ]:
sql_perfume

In [ ]:
sql_perfume.to_csv('data\perfume.csv', index=False)

In [ ]:

sql_perfume_check = pd.read_csv('data\perfume.csv')
print("ROWS in CSV:", len(sql_perfume_check))
print("First 5 rows:\n", sql_perfume_check.head())

In [ ]:

for column in sql_perfume_check.columns:
    sql_perfume_check[column] = sql_perfume_check[column].astype(str)

for column in sql_perfume_check.columns:
    has_special_chars = sql_perfume_check[column].str.contains(r'[^a-zA-Z0-9\s,]', regex=True).any()
    print(f'special_chars in {column}:', has_special_chars)

In [ ]:

for column in sql_perfume_check.columns:
    sql_perfume_check[column] = sql_perfume_check[column].str.replace(r'[^a-zA-Z0-9\s,]', '', regex=True)

In [ ]:
sql_perfume_check.info()

In [ ]:
# cleaned data for SQL and PowerBI
sql_perfume_check.to_csv('sql_perfume_check.csv', sep=',', encoding='utf-8', index=False)

## Product Category Analysis

In [ ]:
category_analysis_full = perfume_df.groupby(['category']).agg({'brand': pd.Series.nunique, 'title' :pd.Series.nunique}).reset_index()
category_analysis_full['sold'] = perfume_df.groupby('category')['sold'].sum().reset_index(drop=True)
category_analysis_full['available'] = perfume_df.groupby('category')['available'].sum().reset_index(drop=True)
category_analysis_full['available_value'] = perfume_df.groupby('category')['available_value'].sum().reset_index(drop=True)

category_analysis_full['total_revenue'] = perfume_df.groupby('category')['total_revenue'].sum().reset_index(drop=True)
category_analysis_full['average_price'] = (category_analysis_full['total_revenue']/category_analysis_full['sold']).round(2)

sold_man_per_category = perfume_df[perfume_df['gender'] == 'man'].groupby('category')['sold'].sum()

category_analysis_full = category_analysis_full.merge(
    sold_man_per_category, 
    how='left', 
    left_on='category', 
    right_index=True, 
    suffixes=('', '_man')
)
category_analysis_full.rename(columns={'sold_man': 'sold_man'}, inplace=True)

category_analysis_full['sold_woman'] = perfume_df[perfume_df['gender'] == 'woman'].groupby('category')['sold'].sum().reset_index(drop=True)
category_analysis_full['sold_man'].fillna(0, inplace=True)
category_analysis_full['sold_woman'].fillna(0, inplace=True)
total_revenue_man_per_category = perfume_df[perfume_df['gender'] == 'man'].groupby('category')['total_revenue'].sum()
total_revenue_man_df = total_revenue_man_per_category.reset_index()
total_revenue_man_df.rename(columns={'total_revenue': 'revenue_man'}, inplace=True)
category_analysis_full = category_analysis_full.merge(
    total_revenue_man_df, 
    how='left', 
    on='category'
)

category_analysis_full['revenue_woman'] = perfume_df[perfume_df['gender'] == 'woman'].groupby('category')['total_revenue'].sum().reset_index(drop=True)
category_analysis_full['revenue_man'].fillna(0, inplace=True)
category_analysis_full=category_analysis_full.sort_values(by='sold', ascending=False)
category_analysis_full = category_analysis_full.reset_index(drop=True)

In [ ]:
display(HTML('<b>Product Category Analysis</b>'))
display(HTML(category_analysis_full.to_html()))

In [ ]:
display(HTML('<b>Sales Volume Analysis</b>'))

categories = category_analysis_full['category']
sold = category_analysis_full['sold']
total_revenue = category_analysis_full['total_revenue']

fig, ax1 = plt.subplots(figsize=(10, 6))

width = 0.35
x = np.arange(len(categories))

bar1 = ax1.bar(x - width/2, sold, width, label='Units Sold', color='lightgreen')


ax2 = ax1.twinx()
bar2 = ax2.bar(x + width/2, total_revenue, width, label='Total Revenue', color='lightblue')


ax1.set_xlabel('Category')
ax1.set_ylabel('Units Sold')
ax2.set_ylabel('Total Revenue (USD)')
ax1.set_title('Total Revenue and Sales Volume by Category')
ax1.set_xticks(x)
ax1.set_xticklabels(categories)


bars = [bar1, bar2]
labels = ['Units Sold', 'Total Revenue']
fig.legend([b[0] for b in bars], labels, loc='upper left')

fig.tight_layout()
plt.show()

### Sales Volume Insights

The "Other" category dominates with 649,501 units sold, contributing to a total revenue of USD 17,899,065, the highest across all categories.
The luxury segment accounts for 340,028 units sold, generating USD 14,157,643 in revenue.
With 257,746 units sold and a total revenue of USD 6,736,388, the mass market caters to a price-sensitive audience, as indicated by the lower average price (USD 26.14).
The niche category has the lowest sales volume at 3,091 units and total revenue of USD 110,922.
Indie brands have the smallest presence, with just 667 units sold and total revenue of USD 28,460.

In [ ]:
display(HTML('<b>Sales Volume and Revenue by Gender</b>'))

categories = category_analysis_full['category']
sold_man = category_analysis_full['sold_man']
sold_woman = category_analysis_full['sold_woman']

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(categories, sold_man, label='Men', color='lightblue')
ax.bar(categories, sold_woman, bottom=sold_man, label='Women', color='pink')

ax.set_xlabel('Category')
ax.set_ylabel('Units Sold')
ax.set_title('Sales Volume by Gender across Categories')
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
for i, category in enumerate(category_analysis_full['category']):
    revenue_man = category_analysis_full['revenue_man'].iloc[i]
    revenue_woman = category_analysis_full['revenue_woman'].iloc[i]
    
    plt.figure(figsize=(5, 5))
    plt.pie([revenue_man, revenue_woman], labels=['Men', 'Women'], autopct='%1.1f%%', colors=['lightblue', 'pink'])
    plt.title(f'Revenue Split by Gender - {category}')
    plt.show()

#### Sales Volume and Revenue by Gender Insights
Sales in the other segment are relatively balanced between men (358,092 units) and women (291,409 units), though men slightly outperform in volume.
Men's products generate USD 10,335,993 in revenue (57.8% of the category's total), while women contribute USD 7,563,072. This gender revenue gap highlights a stronger demand or higher pricing for men's products in this category.

The luxury segment accounts for 340,028 units sold, generating USD 14,157,643 in revenue. A notable skew toward male purchases (241,584 units for men vs. 98,444 for women), with men's sales representing 71% of the total volume. Men's products contribute significantly to the revenue (USD 10,466,504 or 73.9%), while women's products account for USD 3,691,139. The higher average price (USD 41.64) suggests premium positioning, which is well accepted in the market.

With 257,746 units sold and a total revenue of USD 6,736,388, the mass market caters to a price-sensitive audience, as indicated by the lower average price ($26.14). Men lead in both sales volume (161,717 units) and revenue (USD 4,518,345), but the women's segment (96,029 units sold) also holds significant value. Men's products generate approximately 67% of the total revenue, highlighting their stronger foothold in the mass market.

The niche category has the lowest sales volume at 3,091 units and total revenue of USD 110,922. Interestingly, all sales are directed toward women's products, showing no demand for men's offerings. The entire revenue of USD 110,922 comes from women, reflecting a highly targeted product range.

Indie brands have the smallest presence, with just 667 units sold and total revenue of USD 28,460. Slightly more units are sold to women (391 units) compared to men (276 units), but both segments are almost equally represented. Men's products account for USD 15,259 in revenue, while women contribute USD 13,201. The nearly equal revenue split indicates balanced pricing and demand across genders.


In [ ]:
display(HTML('<b>Inventory</b>'))
category_analysis_full['sold_percentage'] = (category_analysis_full['sold'] / category_analysis_full['available']) * 100
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(category_analysis_full['category'], category_analysis_full['available'], width=0.4, label='Available Units', color='lightblue', align='center')
ax2 = ax.twinx()
ax2.plot(category_analysis_full['category'], category_analysis_full['sold_percentage'], color='red', marker='o', label='% Sold')
ax.set_xlabel('Category')
ax.set_ylabel('Available Units')
ax2.set_ylabel('Sold Percentage (%)')
ax.set_title('Available Units vs. Sold Percentage by Category')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Conclusion
The "Other" category drives substantial revenue with balanced sales between genders, but further analysis on inventory management is recommended to optimize stock and sales dynamics.
The luxury category is male-driven, with high unit sales and revenue reflecting strong brand loyalty or effective targeting. However, the relatively low female sales highlight an opportunity for better engagement with the female segment.
The mass market displays a more balanced gender split in sales compared to luxury, but men still dominate. The lower average price reflects the category's competitive nature. Expanding product offerings or promotions in the women's segment could help increase overall market share.
The niche category is entirely women-focused, with low but potentially high-margin sales. The absence of male-targeted products might indicate a missed opportunity or a deliberate brand strategy, but further market research could explore untapped male demand.
Indie brands represent a small but balanced segment in terms of gender distribution. However, low sales and revenue suggest limited market penetration. Opportunities may exist in refining brand positioning or increasing visibility to grow this niche.

#### Overall Insights

Gender Influence: Across categories, men's products generally outperform women's in both sales volume and revenue, especially in the luxury and mass-market segments. This indicates a higher demand for male-oriented products, but also highlights an opportunity for expanding female-targeted offerings.
Revenue vs. Sales: There is a clear correlation between sales volume and revenue in most categories, but disparities in average pricing across categories point to different market strategies. Luxury brands benefit from higher pricing, while mass-market and indie brands focus on affordability.
Inventory Management: High available inventory in categories like "Other" suggests potential inefficiencies. Improving inventory turnover or aligning stock with demand could drive more efficient operations.

## Product Brand Analysis

In [ ]:

brand_grouped_df = perfume_df.groupby('brand').agg({
    'price': 'mean',     
    'available': 'sum',  
    'sold': 'sum',        
    'available_value': 'sum',  
    'total_revenue': 'sum'    
}).reset_index()

brand_grouped_df.rename(columns={'price': 'average_price'}, inplace=True)


In [ ]:
brand_grouped_df.info()

In [ ]:

sold_man_per_brand = perfume_df[perfume_df['gender'] == 'man'].groupby('brand')['sold'].sum().reset_index()
sold_man_per_brand.rename(columns={'sold': 'sold_man'}, inplace=True)

sold_woman_per_brand = perfume_df[perfume_df['gender'] == 'woman'].groupby('brand')['sold'].sum().reset_index()
sold_woman_per_brand.rename(columns={'sold': 'sold_woman'}, inplace=True)

total_revenue_man_per_brand = perfume_df[perfume_df['gender'] == 'man'].groupby('brand')['total_revenue'].sum().reset_index()
total_revenue_man_per_brand.rename(columns={'total_revenue': 'revenue_man'}, inplace=True)

total_revenue_woman_per_brand = perfume_df[perfume_df['gender'] == 'woman'].groupby('brand')['total_revenue'].sum().reset_index()
total_revenue_woman_per_brand.rename(columns={'total_revenue': 'revenue_woman'}, inplace=True)


brand_grouped_df = brand_grouped_df.merge(sold_man_per_brand, on='brand', how='left')
brand_grouped_df = brand_grouped_df.merge(sold_woman_per_brand, on='brand', how='left')
brand_grouped_df = brand_grouped_df.merge(total_revenue_man_per_brand, on='brand', how='left')
brand_grouped_df = brand_grouped_df.merge(total_revenue_woman_per_brand, on='brand', how='left')

brand_grouped_df['sold_man'].fillna(0, inplace=True)
brand_grouped_df['sold_woman'].fillna(0, inplace=True)
brand_grouped_df['revenue_man'].fillna(0, inplace=True)
brand_grouped_df['revenue_woman'].fillna(0, inplace=True)

In [ ]:
brand_grouped_df['category'] = brand_grouped_df['brand'].apply(classify_brand)

In [ ]:
brand_grouped_df

In [ ]:
best_brands = brand_grouped_df.sort_values(by='total_revenue', ascending=False).head(20)
best_brands = best_brands.reset_index(drop=True)
best_brands_man = brand_grouped_df.sort_values(by='revenue_man', ascending=False).head(20)
best_brands_man = best_brands_man.reset_index(drop=True)
best_brands_woman = brand_grouped_df.sort_values(by='revenue_woman', ascending=False).head(20)
best_brands_woman = best_brands_woman.reset_index(drop=True)

In [ ]:
category_brand_count = best_brands.groupby('category')['brand'].nunique().reset_index()

In [ ]:
category_brand_count = category_brand_count.rename(columns = {'brand':'Best in total'})
category_brand_count

In [ ]:
category_brand_count_man = best_brands_man.groupby('category')['brand'].nunique().reset_index()
category_brand_count_man = category_brand_count_man.rename(columns = {'brand':'Best in Man'})
category_brand_count_man

In [ ]:
category_brand_count_woman = best_brands_woman.groupby('category')['brand'].nunique().reset_index()
category_brand_count_woman = category_brand_count_woman.rename(columns = {'brand':'Best in Woman'})
category_brand_count_woman

In [ ]:
categories = pd.merge(category_brand_count,category_brand_count_man,  on='category', how='left')
categories = pd.merge(categories,category_brand_count_woman,  on='category', how='left')

In [ ]:
categories

In [ ]:
display(HTML('<b>Top Brands</b>'))
display(HTML(best_brands.to_html()))
display(HTML('<b>Top Brands by Gender (Man)</b>'))
display(HTML(best_brands_man.to_html()))
display(HTML('<b>Top Brands by Gender (Woman)</b>'))
display(HTML(best_brands_woman.to_html()))

In [ ]:
display(HTML('<b>Sales Volume Top 20 Brands</b>'))

brands = best_brands['brand']
sold_man = best_brands['sold_man']
sold_woman = best_brands['sold_woman']

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(brands, sold_man, label='Men', color='lightblue')
ax.bar(brands, sold_woman, bottom=sold_man, label='Women', color='pink')

ax.set_xlabel('Brand')
ax.set_ylabel('Units Sold')
ax.set_title('Sales Volume by Gender across TOP 20 Brands')
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
display(HTML('<b>Revenue Top 20 Brands</b>'))

revenue_man = best_brands['revenue_man']
revenue_woman = best_brands['revenue_woman']

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(brands, revenue_man, label='Men', color='lightblue')
ax.bar(brands, revenue_woman, bottom=sold_man, label='Women', color='pink')

ax.set_xlabel('Brand')
ax.set_ylabel('Units Sold')
ax.set_title('Revenue by Gender across TOP 20 Brands')
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
total_sales = brand_grouped_df['total_revenue'].sum()

df_sorted = brand_grouped_df.sort_values(by='total_revenue', ascending=False)

df_sorted['cumulative_sales'] = df_sorted['total_revenue'].cumsum()
df_sorted['percentage_of_total'] = (df_sorted['cumulative_sales'] / total_sales)*100

bestsellers_brands = df_sorted[df_sorted['percentage_of_total'] <= 80]
bestsellers_brands = bestsellers_brands.reset_index()
bestsellers_brands = bestsellers_brands.drop(columns=['index'])
bestsellers_brands['category'] = bestsellers_brands['brand'].apply(classify_brand)

In [ ]:
brands_eighty_procent = bestsellers_brands.head(50)

In [ ]:
brands_share = brand_grouped_df[['brand', 'total_revenue']].copy()  # Create a copy to avoid the warning
revenue_sum = brand_grouped_df['total_revenue'].sum()
brands_share.loc[:, 'percentage_of_total'] = (brands_share['total_revenue'] / revenue_sum) * 100
brands_share = brands_share.sort_values(by='percentage_of_total', ascending=False).reset_index(drop=True)

In [ ]:
brands_share.head(20)
brands_share_vis = brands_share.head(45)

In [ ]:
display(HTML('<b>Top brands contributing 80% of the revenue</b>'))
brands_share_vis = brands_share_vis.sort_values(by='percentage_of_total', ascending=False)

brands_share_vis['cumulative_percentage'] = brands_share_vis['percentage_of_total'].cumsum()

fig, ax1 = plt.subplots(figsize=(10, 10))

ax1.barh(brands_share_vis['brand'], brands_share_vis['percentage_of_total'], color='lightblue', label='Percentage of Total Revenue')
ax1.set_xlabel('Percentage of Total Revenue (%)')
ax1.set_ylabel('Brand')

ax2 = ax1.twiny()

ax2.plot(brands_eighty_procent ['percentage_of_total'], brands_eighty_procent['brand'], color='red', marker='o', label='Cumulative Percentage')
ax2.set_xlabel('Cumulative Percentage (%)')

plt.title('Percentage of Total Revenue by Brand with Cumulative Percentage Line')
fig.tight_layout()

plt.show()

The chart reveals that a small number of brands command a significant portion of the total revenue, with Luxury brands leading in both revenue and share of total income. Notably, the 45 brands displayed collectively account for 80% of the total revenue, highlighting their dominance in the market. This underscores the critical role these brands play in the overall financial landscape and their substantial impact on the market's financial performance.

In [ ]:
type_grouped_df = perfume_df.groupby('grouped_category').agg({
    'price': 'mean',     
    'available': 'sum',  
    'sold': 'sum',        
    'available_value': 'sum',  
    'total_revenue': 'sum'    
}).reset_index()

type_grouped_df.rename(columns={'price': 'average_price'}, inplace=True)

In [ ]:
sold_man_per_type = perfume_df[perfume_df['gender'] == 'man'].groupby('grouped_category')['sold'].sum().reset_index()
sold_man_per_type.rename(columns={'sold': 'sold_man'}, inplace=True)

sold_woman_per_type = perfume_df[perfume_df['gender'] == 'woman'].groupby('grouped_category')['sold'].sum().reset_index()
sold_woman_per_type.rename(columns={'sold': 'sold_woman'}, inplace=True)


total_revenue_man_per_type = perfume_df[perfume_df['gender'] == 'man'].groupby('grouped_category')['total_revenue'].sum().reset_index()
total_revenue_man_per_type.rename(columns={'total_revenue': 'revenue_man'}, inplace=True)

total_revenue_woman_per_type = perfume_df[perfume_df['gender'] == 'woman'].groupby('grouped_category')['total_revenue'].sum().reset_index()
total_revenue_woman_per_type.rename(columns={'total_revenue': 'revenue_woman'}, inplace=True)

In [ ]:
type_grouped_df = type_grouped_df.merge(sold_man_per_type, on='grouped_category', how='left')
type_grouped_df = type_grouped_df.merge(sold_woman_per_type, on='grouped_category', how='left')
type_grouped_df = type_grouped_df.merge(total_revenue_man_per_type, on='grouped_category', how='left')
type_grouped_df = type_grouped_df.merge(total_revenue_woman_per_type, on='grouped_category', how='left')
type_grouped_df['sold_man'].fillna(0, inplace=True)
type_grouped_df['sold_woman'].fillna(0, inplace=True)
type_grouped_df['revenue_man'].fillna(0, inplace=True)
type_grouped_df['revenue_woman'].fillna(0, inplace=True)
type_grouped_df = type_grouped_df.sort_values(by='total_revenue', ascending=False).reset_index()
type_grouped_df= type_grouped_df.drop(columns=['index'])

In [ ]:
display(HTML('<b>Sales and Revenue Analysis by Perfume Type and Gender</b>'))
display(HTML(type_grouped_df.to_html()))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.bar(type_grouped_df['grouped_category'], type_grouped_df['sold_man'], label='Sold for Men', color='lightblue')
ax1.bar(type_grouped_df['grouped_category'], type_grouped_df['sold_woman'], bottom=type_grouped_df['sold_man'], label='Sold for Women', color='lightgreen')

ax1.set_xlabel('Perfume Category')
ax1.set_ylabel('Units Sold')
ax1.set_title('Sales Distribution by Gender and Category')
ax1.legend()
ax1.set_xticks(np.arange(len(type_grouped_df['grouped_category'])))
ax1.set_xticklabels(type_grouped_df['grouped_category'], rotation=45)

ax2.bar(type_grouped_df['grouped_category'], type_grouped_df['revenue_man'], label='Revenue from Men', color='lightcoral')
ax2.bar(type_grouped_df['grouped_category'], type_grouped_df['revenue_woman'], bottom=type_grouped_df['revenue_man'], label='Revenue from Women', color='lightpink')

ax2.set_xlabel('Perfume Category')
ax2.set_ylabel('Total Revenue (USD)')
ax2.set_title('Revenue Distribution by Gender and Category')
ax2.legend()
ax2.set_xticks(np.arange(len(type_grouped_df['grouped_category'])))
ax2.set_xticklabels(type_grouped_df['grouped_category'], rotation=45)

plt.tight_layout()
plt.show()

Sales and Revenue Analysis
Eau de Toilette stands out significantly in terms of total sales volume and revenue. The category’s high sales volume (738,461 units) and substantial revenue (USD 23,747,687) highlight its dominant market position. This category’s high performance is driven by a high volume of sales and a considerable revenue contribution from male consumers (USD 19,042,397). The lower average price compared to Eau de Parfum suggests that it appeals to a broader audience, leading to higher sales volume despite the lower per-unit revenue.

Eau de Parfum, while generating less sales volume (326,593 units) compared to Eau de Toilette, commands a higher average price (USD 48.20). This higher average price results in a total revenue of USD 11,337,603, with a significant revenue contribution from female consumers (USD 8,077,727). The data indicates that Eau de Parfum is more premium and preferred among women, reflecting its higher average price and sales revenue.

The 'Other' category, with an average price of USD 38.63, shows moderate performance with a total revenue of USD 2,309,491. However, it is characterized by a substantial male revenue contribution (USD 1,830,556), suggesting that it caters more to male consumers. Despite a lower sales volume, its moderate average price contributes to its total revenue.

Cologne demonstrates a moderate average price (USD 40.41) with lower sales volume (63,565 units) compared to Eau de Toilette and Eau de Parfum. It generates a total revenue of USD 1,360,699, predominantly from male consumers (USD 1,060,702). This indicates that while it has a respectable price point, its overall market impact is limited by its lower sales volume.

Perfume Oil and Body Spray are characterized by their lower average prices (USD 19.55 and USD 17.35, respectively) and low sales volumes (20,919 and 3,085 units, respectively). Perfume Oil, despite its low price, generates more revenue from men (USD 124,590), reflecting a niche market. Body Spray, being the least expensive, generates minimal revenue (USD 43,246) with a relatively even split between male and female consumers.

## Price Analysis

In [ ]:
grouped = perfume_df.groupby('category')['price']
stats = grouped.agg(['mean', 'min', 'max', lambda x: x.mode().iloc[0]])
stats.rename(columns={'<lambda_0>': 'mode'}, inplace=True)
stats_transposed = stats.T
stats_transposed.index = ['Mean', 'Min', 'Max', 'Mode']
stats_transposed = stats_transposed.round(2)

grouped1 = perfume_df.groupby('grouped_category')['price']
stats1 = grouped1.agg(['mean', 'min', 'max', lambda x: x.mode().iloc[0]])
stats1.rename(columns={'<lambda_0>': 'mode'}, inplace=True)
stats_transposed1 = stats1.T
stats_transposed1.index = ['Mean', 'Min', 'Max', 'Mode']
stats_transposed1 = stats_transposed1.round(2)

grouped2 = perfume_df.groupby('gender')['price']
stats2 = grouped2.agg(['mean', 'min', 'max', lambda x: x.mode().iloc[0]])
stats2.rename(columns={'<lambda_0>': 'mode'}, inplace=True)
stats_transposed2 = stats2.T
stats_transposed2.index = ['Mean', 'Min', 'Max', 'Mode']
stats_transposed2 = stats_transposed2.round(2)

combined_stats = pd.concat({
    'Category': stats_transposed,
    'Grouped Category': stats_transposed1,
    'Gender': stats_transposed2
}, axis=1)

display(HTML('<b>Perfume Pricing Metrics by Category, Group, and Gender<b>'))

display(HTML(combined_stats.to_html()))

In [ ]:
display(HTML('<b>Prices and Sales<b>'))

num_bins = 50

hist_values, bin_edges = np.histogram(perfume_df['price'], bins=num_bins)

perfume_df['price_bins'] = pd.cut(perfume_df['price'], bins=bin_edges)

sold_sum = perfume_df.groupby('price_bins')['sold'].sum().reindex(pd.IntervalIndex.from_breaks(bin_edges)).fillna(0).values

fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'lightblue'
ax1.set_xlabel('Price')
ax1.set_ylabel('Frequency', color=color)
hist = ax1.hist(perfume_df['price'], bins=num_bins, alpha=0.5, color=color, label='Price Distribution')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'red'
ax2.set_ylabel('Quantity Sold', color=color)
line = ax2.plot(bin_edges[:-1], sold_sum, color=color, marker='o', linestyle='-', label='Quantity Sold')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Histogram of Perfume Prices with Sold Items Overlay')
fig.tight_layout()  # To prevent clipping of ylabel
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

In [ ]:
display(HTML('<b>Category Price Analysis<b>'))

plt.figure(figsize=(12, 6))

sns.boxplot(x='category', y='price', data=perfume_df)
plt.xlabel('Category')
plt.ylabel('Price (USD)')
plt.title('Price Comparison between categories')
plt.show()

Luxury perfumes have the highest average price at 54.73 USD, followed by Niche (51.12 USD) and Indie (44.42 USD). Massmarket and Other categories have lower mean prices at 35.63 USD and 37.76 USD, respectively. This suggests that Luxury and Niche perfumes are positioned at a higher price point compared to the other categories.

The lowest minimum price is observed in the Other category (1.00 USD), which is significantly lower than the minimum prices in other categories. Massmarket and Indie categories also have relatively low minimum prices at 9.00 USD and 3.00 USD, respectively, indicating a broader range of affordable options.

Luxury perfumes have the highest maximum price at 285.00 USD, showing a potential for premium pricing in this segment. The Other category has the highest variability with a maximum price of 299.00 USD, suggesting a diverse price range in this category. Massmarket and Niche categories have lower maximum prices compared to Luxury and Other.

The most frequently occurring price is relatively consistent across categories, with Luxury and Massmarket categories having a mode of 49.00 USD and 24.00 USD, respectively. This indicates that despite the overall higher average prices in Luxury and Niche categories, the most common price point is lower.

In [ ]:
display(HTML('<b>Gender-Based Price Analysis<b>'))

plt.figure(figsize=(12, 6))

sns.boxplot(x='gender', y='price', data=perfume_df)
plt.xlabel('Gender')
plt.ylabel('Price (USD)')
plt.title('Price Comparison between Man and Woman')
plt.show()

Men's perfumes have a mean price of 45.77 USD, which is slightly higher than Women's perfumes at 39.19 USD. This suggests that Men’s perfumes are generally positioned at a higher price point compared to Women's options.
Minimum Prices:

The minimum price for Men's perfumes is 3.00 USD, while Women's perfumes start at 1.00 USD. This indicates a wider range of very affordable options in the Women's category.
Maximum Prices:

The maximum price for Men’s perfumes is 259.00 USD, while Women's perfumes can go up to 299.00 USD. This suggests that there is a higher ceiling for Women's perfumes, potentially indicating a broader range of premium products.
Mode:

The most frequently occurring price is 29.00 USD for both Men's and Women's perfumes, indicating a common price point that is popular across genders.

In [ ]:
display(HTML('<b>Product Type Price Analysis<b>'))

plt.figure(figsize=(12, 6))

sns.boxplot(x='grouped_category', y='price', data=perfume_df)
plt.xlabel('Product type')
plt.ylabel('Price (USD)')
plt.title('Price Comparison between Product Types')
plt.show()


Body Spray has the lowest mean price at 17.35 USD, while Eau de Parfum (48.20 USD) and Cologne (40.41 USD) are priced higher, showing that Eau de Parfum and Cologne are positioned at a premium compared to Body Spray and Perfume Oil.

Other products have the lowest minimum price of 1.00 USD, indicating the availability of very affordable options in this category. Body Spray also has a low minimum price (6.00 USD), suggesting that it is generally more affordable.
Maximum Prices:

Eau de Parfum and Cologne have the highest maximum prices, 285.00 USD and 129.00 USD, respectively, showing that these categories can command higher prices. Body Spray and Perfume Oil have lower maximum prices, reflecting a more affordable range.

The most common price point varies by category, with Cologne and Eau de Parfum having modes of 29.00 USD, reflecting their popularity at that price. Body Spray and Perfume Oil have lower modes, which are consistent with their generally lower price ranges.

## Geographic analysis

In [ ]:

location_grouped_df = perfume_df.groupby('itemLocation').agg({
    'price': 'mean',     
    'available': 'sum',  
    'sold': 'sum',        
    'available_value': 'sum',  
    'total_revenue': 'sum'    
}).reset_index()

brand_grouped_df.rename(columns={'price': 'average_price'}, inplace=True)

sold_man_per_location = perfume_df[perfume_df['gender'] == 'man'].groupby('itemLocation')['sold'].sum().reset_index()
sold_man_per_location.rename(columns={'sold': 'sold_man'}, inplace=True)

sold_woman_per_location = perfume_df[perfume_df['gender'] == 'woman'].groupby('itemLocation')['sold'].sum().reset_index()
sold_woman_per_location.rename(columns={'sold': 'sold_woman'}, inplace=True)

total_revenue_man_per_location = perfume_df[perfume_df['gender'] == 'man'].groupby('itemLocation')['total_revenue'].sum().reset_index()
total_revenue_man_per_location.rename(columns={'total_revenue': 'revenue_man'}, inplace=True)

total_revenue_woman_per_location = perfume_df[perfume_df['gender'] == 'woman'].groupby('itemLocation')['total_revenue'].sum().reset_index()
total_revenue_woman_per_location.rename(columns={'total_revenue': 'revenue_woman'}, inplace=True)

location_grouped_df = location_grouped_df.merge(sold_man_per_location, on='itemLocation', how='left')
location_grouped_df = location_grouped_df.merge(sold_woman_per_location, on='itemLocation', how='left')
location_grouped_df = location_grouped_df.merge(total_revenue_man_per_location, on='itemLocation', how='left')
location_grouped_df = location_grouped_df.merge(total_revenue_woman_per_location, on='itemLocation', how='left')

location_grouped_df['sold_man'].fillna(0, inplace=True)
location_grouped_df['sold_woman'].fillna(0, inplace=True)
location_grouped_df['revenue_man'].fillna(0, inplace=True)
location_grouped_df['revenue_woman'].fillna(0, inplace=True)
location_grouped_df = location_grouped_df.round(2)
location_grouped_df = location_grouped_df.sort_values(by='total_revenue', ascending=False).reset_index(drop=True)

In [ ]:
location_grouped_df['cumulative_sales'] = location_grouped_df['total_revenue'].cumsum()
location_grouped_df['percentage_of_total'] = (location_grouped_df['cumulative_sales'] / total_sales)*100

bestsellers_items = location_grouped_df[location_grouped_df['percentage_of_total'] <= 90]
bestsellers_items = bestsellers_items.reset_index()
bestsellers_items = bestsellers_items.drop(columns=['index'])
bestsellers_items['percentage_of_total'] = (bestsellers_items['total_revenue']/total_sales)*100
bestsellers_items = bestsellers_items.round(2)

In [ ]:
bestsellers_items_top = bestsellers_items 

In [ ]:
display(HTML('<b>Top items contributing 90% of the revenue</b>'))
display(bestsellers_items_top)
bestsellers_items_top['cumulative_percentage'] = bestsellers_items_top['percentage_of_total'].cumsum()

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.barh(bestsellers_items_top['itemLocation'], bestsellers_items_top['total_revenue'], color='lightblue', label='Total Revenue')
ax1.set_xlabel('Total Revenue')
ax1.set_ylabel('Item Location')

ax2 = ax1.twiny()
ax2.plot(bestsellers_items_top['cumulative_percentage'], bestsellers_items_top['itemLocation'], color='red', marker='o', label='Cumulative Percentage')
ax2.set_xlabel('Cumulative Percentage (%)')

plt.title('Percentage of Total Revenue by Item Location with Cumulative Percentage Line')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

fig.tight_layout()

plt.show()

In [ ]:
locations = [
    "Hackensack, New Jersey, United States",
    "Dallas, Texas, United States",
    "Katy, Texas, United States",
    "Edison, New Jersey, United States",
    "Houston, Texas, United States"
]


filtered_df = perfume_df[perfume_df['itemLocation'].isin(locations)]

inventory_pivot_category = filtered_df.pivot_table(
    index='itemLocation', 
    columns='category', 
    values='available_value', 
    aggfunc='sum',
    fill_value=0
)

inventory_pivot_grouped_category = filtered_df.pivot_table(
    index='itemLocation', 
    columns='grouped_category', 
    values='available_value', 
    aggfunc='sum',
    fill_value=0
)


inventory_df = pd.concat([inventory_pivot_category, inventory_pivot_grouped_category], axis=1)

inventory_df = inventory_df.reset_index()
inventory_df = inventory_df.rename(columns = {'Indie':'Indie_inv', 
                                              'Luxury' : 'Luxury_inv',	
                                              'Massmarket':'Massmarket_inv',	
                                              'Niche':'Niche_inv',	
                                              'Other_category':'Other_Category_inv',	
                                              'Body Spray':'Body Spray_inv',
                                              'Cologne':'Cologne_inv',
                                              'Eau de Parfum':'Eau de Parfum_inv',
                                              'Eau de Toilette':
                                              'Eau de Toilette_inv',
                                              'Other_product_type':'Other_product_type_inv'})

In [ ]:
sales_pivot_category = filtered_df.pivot_table(
    index='itemLocation', 
    columns='category', 
    values='total_revenue', 
    aggfunc='sum',
    fill_value=0
)

sales_pivot_grouped_category = filtered_df.pivot_table(
    index='itemLocation', 
    columns='grouped_category', 
    values='total_revenue', 
    aggfunc='sum',
    fill_value=0
)


sales_df = pd.concat([sales_pivot_category, sales_pivot_grouped_category], axis=1)


sales_df = sales_df.reset_index()
sales_df = sales_df.rename(columns = {'Indie':'Indie_rev', 
                                      'Luxury' : 'Luxury_rev',	
                                      'Massmarket':'Massmarket_rev',	
                                      'Niche':'Niche_rev',	
                                      'Other_category':'Other_Category_rev',	
                                      'Body Spray':'Body Spray_rev',
                                      'Cologne':'Cologne_rev',
                                      'Eau de Parfum':'Eau de Parfum_rev',
                                      'Eau de Toilette':'Eau de Toilette_rev',
                                      'Other_product_type':'Other_product_type_rev'})

In [ ]:
inv_eff_base = inventory_df.merge(sales_df, on='itemLocation', how='left')
inv_eff_base['Indie_IUR'] = inv_eff_base['Indie_rev']/inv_eff_base['Indie_inv']

inv_eff_base['Luxury_IUR'] = inv_eff_base['Luxury_rev']/inv_eff_base['Luxury_inv']
inv_eff_base['Massmarket_IUR'] = inv_eff_base['Massmarket_rev']/inv_eff_base['Massmarket_inv']
inv_eff_base['Niche_IUR'] = inv_eff_base['Niche_rev']/inv_eff_base['Niche_inv']
inv_eff_base['Other_Category_IUR'] = inv_eff_base['Other_Category_rev']/inv_eff_base['Other_Category_inv']
inv_eff_base['Body Spray_IUR'] = inv_eff_base['Body Spray_rev']/inv_eff_base['Body Spray_inv']
inv_eff_base['Cologne_IUR'] = inv_eff_base['Cologne_rev']/inv_eff_base['Cologne_inv']
inv_eff_base['Eau de Parfum_IUR'] = inv_eff_base['Eau de Parfum_rev']/inv_eff_base['Eau de Parfum_inv']
inv_eff_base['Eau de Toilette_IUR'] = inv_eff_base['Eau de Toilette_rev']/inv_eff_base['Eau de Toilette_inv']
inv_eff_base['Other_product_type_IUR'] = inv_eff_base['Other_product_type_rev']/inv_eff_base['Other_product_type_inv']
inv_eff_base = inv_eff_base.fillna({
    'Indie_IUR':0,
    'Luxury_IUR':0,
    'Massmarket_IUR':0,
    'Niche_IUR':0,
    'Other_Category_IUR':0,
    'Body Spray_IUR':0,
    'Cologne_IUR':0,
    'Eau de Parfum_IUR':0,
    'Eau de Toilette_IUR':0,
    'Other_product_type_IUR':0,
})
inv_eff_base=inv_eff_base.round(2)

In [ ]:
iur = inv_eff_base[['itemLocation',
                    'Indie_IUR',
    'Luxury_IUR',
    'Massmarket_IUR',
    'Niche_IUR',
    'Other_Category_IUR',
    'Body Spray_IUR',
    'Cologne_IUR',
    'Eau de Parfum_IUR',
    'Eau de Toilette_IUR',
    'Other_product_type_IUR']]

In [ ]:
display(HTML('<b>Inventory Top Locations<b>'))
display(inventory_df)
display(HTML('<b>Sales Top Locations<b>'))
display(sales_df)
display(HTML('<b>Inventory Utilization Ratios (IUR)<b>'))
display(iur)

- **Hackensack**:
  - Strong performance in **Luxury (653.04)** and **Massmarket (1211.00)** IURs indicates a highly efficient inventory turnover.
  - The **Other Category IUR (452.56)** suggests good management, while the **Niche category (inf)** points to possible inventory issues or lack of sales data.

- **Dallas**:
  - Reasonable IURs in **Massmarket (28.65)** and **Other Category (15.91)** demonstrate effective inventory management, but **Indie (0.0)** and **Niche (0.00)** categories are underperforming, indicating potential missed opportunities.

- **Katy**:
  - The **Eau de Parfum IUR (38.37)** is notable, but overall low sales and inventory levels suggest that more aggressive marketing or better stock management is necessary.

- **Edison**:
  - **Luxury IUR (29.82)** is promising, while **Massmarket IUR (16.73)** indicates moderate performance. However, significant inefficiencies in the **Cologne (inf)** category suggest inventory problems.

- **Houston**:
  - Low IURs across most categories, especially **Luxury (10.64)** and **Niche (0.33)**, point to inefficiencies in inventory planning and demand forecasting. More strategic inventory management is crucial here.

In [ ]:
bestsellers_items_top_stock = bestsellers_items_top.merge(inventory_df, on='itemLocation', how='left')

In [ ]:
bestsellers_items_top_stock = bestsellers_items_top_stock.merge(sales_df, on='itemLocation', how='left')
bestsellers_items_top_stock = bestsellers_items_top_stock.merge(iur, on='itemLocation', how='left')

### Inventory and Sales Performance Analysis

#### Summary of Key Metrics

1. **Sales Performance**:
   - **Total Sold**: The highest sales volume is seen in **Hackensack** (617,390), followed by **Dallas** (435,567). **Katy** and **Edison** show significantly lower sales figures, indicating potential issues in either demand or inventory levels.
   - **Total Revenue**: Hackensack also leads in total revenue ($21,537,042), with Dallas following at $11,085,194. The lower revenue in Katy ($908,275) and Edison ($688,948) points to a need for improvement.

2. **Inventory Levels**:
   - **Available Stock**: **Dallas** has the highest available stock (23,428), suggesting good inventory management. In contrast, **Katy** has only 395 items available, which may not be sufficient to meet demand based on its sales volume.
   - **Available Value**: The highest available value is in **Dallas** ($649,479), which correlates with its sales figures. However, Hackensack's lower available stock still yields substantial revenue due to effective sales.

### Conclusion
The data illustrates clear disparities in inventory utilization and sales performance across different locations. By implementing strategic adjustments and focusing on underperforming categories, overall sales efficiency and revenue can be improved significantly.